In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn import linear_model
from sklearn import datasets
from sklearn import preprocessing, utils
from sklearn.metrics import  make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
df_av = pd.read_csv("avila.txt", header=None)
idx=  np.array(df_av.iloc[:,-1])
y = np.zeros((len(idx)), dtype=np.int32)
y[idx=='A'] = 1

#divide data into x and y
x = df_av.iloc[:,:-1]

#seeing %pos in the data
pos = y.sum()
neg = len(y) - pos

percent_pos = round(pos/len(y)*100, 2)
percent_neg = round(neg/len(y)*100, 2)

percent_pos

41.09

In [3]:
x_std = preprocessing.scale(x).astype(np.float32)
y_std = y.astype(np.float32)

In [7]:
x_std.shape
# y_std.shape

(10430, 10)

In [8]:
result_rf = []
for trial in range(5):
    #splitting into training and testing
    x_train, x_test, y_train, y_test = train_test_split(x_std, y_std, train_size = 5000, random_state=trial,
                                                    stratify = y_std, shuffle=True)

    #encode training and testing data != continuous
    encode_ytr = preprocessing.LabelEncoder()
    ytr_encoded = encode_ytr.fit_transform(y_train)
    
    encode_yt = preprocessing.LabelEncoder()
    yt_encoded = encode_yt.fit_transform(y_test)
    
    criterion_list = ["gini", "entropy"]
    max_list = [1, 2, 4, 6, 8]
    alg = RandomForestClassifier(n_estimators=1024)
    param_grid = {'criterion':criterion_list, "max_features": max_list}
    scoring_list = ['accuracy', 'precision', 'f1', 'roc_auc', 'recall']
    scoring_func_dict = {'acc':accuracy_score, 'AUC':roc_auc_score,
                        'prec':precision_score, 'recall':recall_score, 'f1':f1_score}
    scoring_dict = {k:make_scorer(v) for k,v in scoring_func_dict.items()}
    
    search_result = GridSearchCV (alg, param_grid, scoring=scoring_dict, refit=False, cv=5, n_jobs=-1)
    search_result.fit(x_train, ytr_encoded)
    
    res_one_trial = []
    for k in scoring_dict.keys():
        
        mean_test_metric = search_result.cv_results_['mean_test_'+k]
        best_idx = np.argmax(mean_test_metric)
        best_params = search_result.cv_results_['params'][best_idx]
        alg_new_5000 = RandomForestClassifier(n_estimators=1024, **best_params)
        alg_new_5000.fit(x_train, ytr_encoded)
        
        res_test = alg_new_5000.predict(x_test)
        
        res_one_model = []
        for k_metric, func_metric in scoring_func_dict.items():
            metric_value = func_metric(res_test, yt_encoded)
            res_one_model.append(metric_value)

        res_one_trial.append(res_one_model)
    
    result_rf.append(res_one_trial)
result_rf = np.array(result_rf)
print(result_rf.shape)
print(np.mean(result_rf, axis=0))

(5, 5, 5)
[[0.98169429 0.982013   0.97158225 0.98366808 0.97758202]
 [0.98132597 0.98157799 0.9714926  0.98286577 0.97713753]
 [0.98154696 0.98177792 0.97194083 0.98296305 0.9774137 ]
 [0.98151013 0.98173406 0.97194083 0.9828773  0.97737086]
 [0.98180479 0.98196909 0.97274765 0.98279925 0.97773998]]


In [9]:
print([k for  k in scoring_dict.keys()])
print([k for k in scoring_func_dict.keys()])

['acc', 'AUC', 'prec', 'recall', 'f1']
['acc', 'AUC', 'prec', 'recall', 'f1']
